# Function Calling In LangChain

In [ ]:
import json
import boto3
import panel as pn  # GUI

from typing import List
from pydantic import BaseModel, Field
from langchain_aws import ChatBedrock

bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
)


In [ ]:
#!pip cache purge
#!pip uninstall --yes pydantic langchain langchain-core langchain-text-splitters langchain-community langchain-aws langchain-anthropic jupyter-ai jupyter-ai-magics amazon-sagemaker-jupyter-ai-q-developer
#!pip install pydantic==2.9.2
#!pip install jupyter-ai==2.24.0
#!pip install langchain==0.3.1
    #0.2.16
#!pip install langchain-anthropic
#!pip install langchain-aws


## Pydantic Syntax

Pydantic data classes are a blend of Python's data classes with the validation power of Pydantic. 

They offer a concise way to define data structures while ensuring that the data adheres to specified types and constraints.

In standard python you would create a class like this:

In [ ]:
class User:
    def __init__(self, name: str, age: int, email: str):
        self.name = name
        self.age = age
        self.email = email

In [ ]:
foo = User(name="Joe",age=32, email="joe@gmail.com")

In [ ]:
foo.name

In [ ]:
foo = User(name="Joe",age="bar", email="joe@gmail.com")

In [ ]:
foo.age

In [ ]:
class pUser(BaseModel):
    name: str
    age: int
    email: str

In [ ]:
foo_p = pUser(name="Jane", age=32, email="jane@gmail.com")

In [ ]:
foo_p.name

**Note**: The next cell is expected to fail.

In [ ]:
foo_p = pUser(name="Jane", age="bar", email="jane@gmail.com")

In [ ]:
class Class(BaseModel):
    students: List[pUser]

In [ ]:
obj = Class(
    students=[pUser(name="Jane", age=32, email="jane@gmail.com")]
)

In [ ]:
obj

## Pydantic to Anthropic tool definition

In [ ]:
class WeatherSearch(BaseModel):
    """Call this with an airport code to get the weather at that airport. Make a best guess at the airport code."""
    airport_code: str = Field(description="airport code to get weather for")

In [ ]:
from langchain_anthropic import ChatAnthropic
from langchain_aws.function_calling import convert_to_anthropic_tool


In [ ]:
weather_function = convert_to_anthropic_tool(WeatherSearch)

In [ ]:
weather_function

In [ ]:
class WeatherSearch1(BaseModel):
    airport_code: str = Field(description="airport code to get weather for")

**Note**: The next cell does not cause an error with the anthropic converter, but leaves an empty description.

In [ ]:
convert_to_anthropic_tool(WeatherSearch1)

In [ ]:
class WeatherSearch2(BaseModel):
    """Call this with an airport code to get the weather at that airport"""
    airport_code: str

In [ ]:
convert_to_anthropic_tool(WeatherSearch2)

In [ ]:
model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
    model_kwargs=dict(temperature=0)
)

In [ ]:
model.invoke("what is the weather in SF today?", tools=[weather_function])

In [ ]:
model_with_function = model.bind(tools=[weather_function])

In [ ]:
model_with_function.invoke("what is the weather in sf?")

## Alternate syntax

We can skip converting and just bind a plain pydantic model

In [ ]:
alternate_model_with_function = model.bind_tools(tools=[WeatherSearch])

In [ ]:
alternate_model_with_function.invoke("what is the weather in sf?")

## Forcing it to use a tool

We can force the model to use a function/tool


In [ ]:
model_with_forced_function = model.bind_tools(tools=[WeatherSearch], tool_choice={"type": "tool", "name": "WeatherSearch"})

In [ ]:
model_with_forced_function.invoke("what is the weather in sf?")

In [ ]:
model_with_forced_function.invoke("hi!")

## Using in a chain

We can use this model bound to function in a chain as we normally would

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("user", "{input}")
])

In [ ]:
chain = prompt | model_with_function

In [ ]:
chain.invoke({"input": "what is the weather in sf?"})

## Using multiple functions

Even better, we can pass a set of function and let the LLM decide which to use based on the question context.

In [ ]:
class ArtistSearch(BaseModel):
    """Call this to get the names of songs by a particular artist"""
    artist_name: str = Field(description="name of artist to look up")
    n: int = Field(description="number of results")

In [ ]:
tools = [WeatherSearch, ArtistSearch]

In [ ]:
model_with_tools = model.bind_tools(tools=tools)

In [ ]:
model_with_tools.invoke("what is the weather in sf?")

In [ ]:
model_with_tools.invoke("what are three songs by taylor swift?")

In [ ]:
model_with_tools.invoke("hi!")